In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain.chains import create_extraction_chain_pydantic
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.pydantic_v1 import BaseModel

In [7]:

from __future__ import annotations

from enum import Enum
from typing import Optional, List

from pydantic import BaseModel, Field, ValidationError

import json


# Pydantic Models used for extraction


class ContactType(Enum):
    Phone = 'Phone'
    Text = 'Text'


class CustomerNeed(Enum):
    """ Primary reason why the customer is calling """
    Complete_purchase_support = 'Complete purchase support'
    Technical_support = 'Technical or Product support'
    Membership_management = 'Membership management'
    Help_with_appointment_scheduling = 'Help with appointment scheduling'
    Other = 'Other'


class EmployeeResponse(Enum):
    """ How did the agent on the call ultimately help the customer """
    Fixed_or_resolved_in_the_moment = 'Fixed or resolved in the moment'
    Added_protection_plan_to_product = 'Added protection plan to product'
    Engaged_manager_or_support = 'Engaged manager or support'
    Gave_recommendation_or_advice = 'Gave recommendation or advice'
    Started_remote_support_session = 'Started remote support session'
    Unable_to_resolve_to_satisfaction = 'Unable to resolve to satisfaction'
    Canceled_membership = 'Canceled membership'
    Explained_membership_benefits = 'Explained membership benefits'
    Referred_to_other_team___unable_to_help = 'Referred to other team - unable to help'
    Renewed_membership = 'Renewed membership'
    Rescheduled_appointment = 'Rescheduled appointment'
    Other = 'Other'


class GiftCard(BaseModel):
    giftCardOffered: bool = Field(
        ...,
        description='Was a gift card offered to the customer?',
        title='Giftcardoffered',
    )
    giftCardAmount: str = Field(
        ...,
        description='Dollar amount of the gift card offered',
        title='Giftcardamount',
    )
    giftCardAccepted: bool = Field(
        ...,
        description='If gift card was offered, did the customer accept the gift card?',
        title='Giftcardaccepted',
    )


class SentimentClassifications(Enum):
    Positive = 'Positive'
    Negative = 'Negative'
    Neutral = 'Neutral'


class CustomerSentiment(BaseModel):
    sentiment: SentimentClassifications = Field(
        ..., description="Classify the customer's sentiment at the time"
    )
    reasoning: str = Field(
        ...,
        description="provide your reasoning for why you classified the customer's sentiment as you did",
        title='Reasoning',
    )


class CaseSchema(BaseModel):
    """ Information about the Customer's Case Conversation """
    summary: str = Field(
        ...,
        description='A detailed summary of the entire call transcript. Be sure to include why the customer was calling and if the customers issue was resolved and any other details that are worth mentioning. Do not use the customer or agents actual name in the summary, just refer to them as either "Agent" or "Customer"!',
        title='Summary',
    )
    orderNumber: str = Field(
        ..., description="The customer's order number", title='Ordernumber'
    )
    productSKU: str = Field(
        ...,
        description='The product SKU. The SKU is a GUID that is typically a string of digits. DO NOT assign a non-GUID value to productSKU!',
        title='Productsku',
    )
    driver: Optional[str] = Field(
        None,
        description="A name or description that could be used to identify the individual delivering and/or doing service at the customer's house",
        title='Driver',
    )
    photos: Optional[bool] = Field(
        None,
        description="where photos captured at the customer's house showing damage or product condition?",
        title='Photos',
    )
    agentCall: Optional[bool] = Field(
        None,
        description="Did the agent or delivery person make a call while at the customer's house?",
        title='Agentcall',
    )
    contactType: ContactType = Field(..., description='Contact Channel')
    productSafetyFlag: bool = Field(
        ..., description='Is there a product safety concern?', title='Productsafetyflag'
    )
    customerNeed: CustomerNeed = Field(..., title='Customer Need')
    employeeResponse: EmployeeResponse = Field(..., title='Employee Response')
    customerSentimentGoingIn: CustomerSentiment = Field(
        ...,
        description="How would you classify the customer's sentiment at the beginning of the call",
    )
    customerSentimentGoingOut: CustomerSentiment = Field(
        ...,
        description="How would you classify the customer's sentiment at the end of the call",
    )
    agentName: Optional[str] = Field(
        None, description='The name of the agent', title='Agentname'
    )
    customerName: Optional[str] = Field(
        None, description='The name of the customer', title='Customername'
    )
    giftCards: Optional[GiftCard] = Field(
        None,
        description='Were gift cards offered and if so, how much, and were they accepted by the customer?',
    )
# End Pydantic Models


In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser

#model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract the relevant information, if not explicitly provided do not guess. 
         If there is no value do not substitute a value in its place (e.g. "N/A", ""), just leave the value as None.
         """),
    ("human", "{input}")
])

case_extraction_functions = [convert_pydantic_to_openai_function(CaseSchema)]
extraction_model = model.bind(
    functions=case_extraction_functions, 
    function_call={"name":"CaseSchema"}
)
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [5]:


call_transcript_1 = {
    "transcript": [{
        "source": "agent",
        "timestamp": "2024-01-17T18:26:17.671956",
        "message": "Hi Thanks for calling geek squad. My name is Valerie before we start. May I please have your full name and your phone number."
    }, {
        "source": "customer",
        "timestamp": "2024-01-17T18:26:27.609811",
        "message": "No. Yes, I just I'm sorry I think I was in the wrong department I need to talk to somebody about printer I bought, the order number is 806185814326."
    }, {
        "source": "agent",
        "timestamp": "2024-01-17T18:26:33.117753",
        "message": "Uh Yes, I am from computing services, but what do you need? I'm sorry I cannot hear you properly."
    }, {
        "source": "customer",
        "timestamp": "2024-01-17T18:26:37.875478",
        "message": "I need to know if you sell a specific type of printer in."
    }, {
        "source": "agent",
        "timestamp": "2024-01-17T18:26:41.890700",
        "message": "Okay, I am going to provide you the phone number of the right department. Okay?"
    }, {
        "source": "customer",
        "timestamp": "2024-01-17T18:26:48.074268",
        "message": "No cause I already called and was on hold for 12 minutes and I got disconnected. So can you connect me."
    }, {
        "source": "agent",
        "timestamp": "2024-01-17T18:26:52.816488",
        "message": "Uh, yes, for sure. Thank you so much for calling geek squad and have a wonderful day."
    }, {
        "source": "customer",
        "timestamp": "2024-01-17T18:26:54.107702",
        "message": "Thank you."
    }, {
        "source": "agent",
        "timestamp": "2024-01-17T18:26:52.816488",
        "message": "For your trouble, I would like to offer you a $20 gift card, would that be acceptable?"
    }, {
        "source": "customer",
        "timestamp": "2024-01-17T18:26:54.107702",
        "message": "No, that is ok.."
    }, {
        "source": "agent",
        "timestamp": "2024-01-17T18:27:39.057198",
        "message": "All agents are busy at this time. Please hold."
    }]
}

call_transcript_2 = {
  "transcript": [{
    "source": "agent",
    "timestamp": "2024-01-17T18:29:40.381302",
    "message": "Hello Thank you for calling best buy my name is Stewart, may I get your name and phone number."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:29:49.338916",
    "message": "Hi Stewart, my name is Ellen E. L. L E. N. And my phone number is 3144824330"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:29:54.733837",
    "message": "Alright, got your account pulled up, how can I help you today."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:04.567761",
    "message": "Yeah, so I am upset, um, we've bought a fridge (order number BBY01-732395814518). Through you guys, um, not yesterday, but today for yesterday. And then I received a text yesterday, saying, um, the product was delayed by like."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:09.908193",
    "message": "Four months, um, and that I needed to reschedule but when I clicked the link."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:15.516324",
    "message": "Um, it was very confusing. So, uh, I just wanted to call and"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:30:17.119571",
    "message": "Okay."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:17.507810",
    "message": "See what's on."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:23.333945",
    "message": "Yeah, no worries. Thank you."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:30:24.963298",
    "message": "Yeah, sorry to hear about that. Let me pull up your order to see what's going on, okay?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:28.593732",
    "message": "Yes."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:30:30.535610",
    "message": "No problem. Okay? Is it for the GE fridge."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:30:36.615152",
    "message": "Yeah, it looks like. This one's coming direct from the vendor usually Those are hit or miss."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:39.200744",
    "message": "Okay, um"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:30:41.947497",
    "message": "Oh, and then smart dish special order, too."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:30:50.235495",
    "message": "Mm Let's see, I'm pulling up the dates when it's saying we should have it in stock."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:30:50.321401",
    "message": "Yeah, it said so, my Um delivery day was."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:31:00.729881",
    "message": "Is like the 15th, or the 13th or something of February, or maybe it was the 8th. I don't know early, February, um, but then the text message I got said."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:31:06.291983",
    "message": "Which is very late, um"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:31:09.099324",
    "message": "Is that the case, or"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:31:09.651951",
    "message": "Yeah, I?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:31:22.336393",
    "message": "Yeah, that's"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:31:25.493958",
    "message": "I pulled up the The date and it is kind of absurd it says. It's not supposed to be in stock until May 2nd, which is way far out, so I I would recommend?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:31:28.914220",
    "message": "Okay."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:31:32.598238",
    "message": "Probably going back in store and looking for another unit as bad as it sounds, unless That's like the unit?"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:31:35.701207",
    "message": "Okay."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:31:43.635481",
    "message": "Doesn't Yeah, that's not an option, um, that isn't an option, um, so it is may fit, uh, may 2nd that I won't I won't be able to get the fridge until May 2nd? That's confirmed."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:31:45.263154",
    "message": "Yeah."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:31:48.917179",
    "message": "2nd 3rd and 4th Those are the soonest"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:31:57.940965",
    "message": "Okay, um, so if we did still go through you, guys. Then, um, would you suggest that I schedule it after the 2nd the install and delivery?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:07.114804",
    "message": "Okay."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:12.123376",
    "message": "Okay."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:15.354302",
    "message": "Um"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:32:18.599837",
    "message": "No basically the date that it's giving me is when we're supposed to get it in, so that usually means they get it in the day before like at night, so they should get it on May 1st, and have it ready on May 2nd. It'll let us schedule it for whatever that is."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:34.740383",
    "message": "And, uh, since that is so different than what, when we bought it the day before, um, I I'm not sure that's gonna work for us, um, if it doesn't work for us? Are we able to return it and get full refund back, um, and buy it through a different a different company."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:32:36.443703",
    "message": "Yeah, absolutely."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:40.288319",
    "message": "Okay."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:49.097388",
    "message": "Okay, oh, it looks like."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:32:50.906590",
    "message": "Yeah, basically that order is currently ticketed for you, guys. And so you have all the way up until May 2nd to do any modifications cancellations and things like that."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:55.679731",
    "message": "I just got a new email I don't know if you just did something, but it says? Your appointment with our service partner is 52"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:32:57.494220",
    "message": "Okay, so you just"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:32:59.679629",
    "message": "Yeah."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:33:00.570214",
    "message": "Okay, thank you."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:33:02.416846",
    "message": "I went ahead and pushed it through."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:33:06.735276",
    "message": "No problem."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:33:09.588079",
    "message": "Um, okay, perfect. So, uh, I'll subscribe to my husband just cause we're doing? Oh, I just gotta make sure it fits within our time line I don't think it will. But if it's"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:33:19.346998",
    "message": "I I'm not 100 percent sure. So, um, if I do need to cancel it and get a refund Um, what I just call this number back or"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:33:21.154769",
    "message": "Um How would I go, okay?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:33:22.613550",
    "message": "Okay, fabulous."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:33:24.862095",
    "message": "Yeah, same number."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:33:25.339313",
    "message": "Thank you so much, I really appreciate for all the help."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:33:29.023411",
    "message": "You too? Thanks so much. Bye. Bye."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:33:31.030746",
    "message": "Oh no problem you have a great day. Okay?"
  }]
}

call_transcript_3 = {
  "transcript": [{
    "source": "agent",
    "timestamp": "2024-01-17T18:25:57.082914",
    "message": "Hello, thank you very much for calling best buy my name is lily Can I please get started with the phone number so I can assist you. Today, please."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:26:02.082232",
    "message": "Um, hi, Lily, it's 3104698373"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:26:10.389869",
    "message": "Okay, so I have 3104698373 Is that a good callback number."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:26:17.211298",
    "message": "Okay, perfect, I was able to find an account under that phone number, what is your first and last name, please?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:26:18.442956",
    "message": "Julie schwackenwall"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:26:26.663228",
    "message": "Okay, perfect I do see an account under your first and last name and to verify your information, what is your email address, please."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:26:28.176892",
    "message": "Julie schwak @gmail.com"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:26:38.360611",
    "message": "Okay, perfect. Thank you very much for verifying your information, and Thank you so much for being my best buy member with us. How can I take care of you today."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:26:48.724379",
    "message": "I Um scheduled an appointment earlier this morning for tomorrow and I just I haven't heard, um, about a time for tomorrow. Yet, so I just I wanted to check in?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:26:59.021418",
    "message": "Repair."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:26:59.518374",
    "message": "Okay, I can definitely take a look and see if an error that window has generated for the appointment. That is supposed to be for tomorrow is this for repair or delivery?"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:27:08.654810",
    "message": "Okay, Let me pull up my repair system give me one second Thank you so much."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:27:24.382129",
    "message": "Okay, let's see okay 3104698373"
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:27:50.773552",
    "message": "So I do see that the appointment is scheduled. Let me take a look that at the activity lock and It looks like the appointment was created. Today, um, so so far. It was created today at 1026 am It looks like it was."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:27:55.988475",
    "message": "Schedule, but right now, the narrow down window has not generated yet."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:27:58.312654",
    "message": "Okay, when will I hear about that."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:28:11.113314",
    "message": "As soon as the agent is able is able to confirm that our appointment and narrow that window, will get created so I'm hoping that you will get an error that window before the end of the day today?"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:28:11.865632",
    "message": "Well, don't I? If if the"
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:28:14.550307",
    "message": "Okay."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:28:14.576217",
    "message": "The latest could be seven."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:28:15.873375",
    "message": "Okay."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:28:17.488892",
    "message": "Seven PM eight PM."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:28:18.219910",
    "message": "Great. Thank you."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:28:25.269661",
    "message": "No thanks, lily."
  }, {
    "source": "agent",
    "timestamp": "2024-01-17T18:28:26.115720",
    "message": "No problem, definitely Is there anything else that you want me to take a look at, for you today."
  }, {
    "source": "customer",
    "timestamp": "2024-01-17T18:28:29.542338",
    "message": "You, too, bye. Bye."
  }]
}

call_transcript_dict = {
    "1": call_transcript_1,
    "2": call_transcript_2,
    "3": call_transcript_3
}


In [9]:
import json

# Run
json_output = extraction_chain.invoke({"input": call_transcript_dict["1"]})

# the model sometimes wants to use empty strings instead of None
for key, value in json_output.items():
    if isinstance(value, str) and value == '':
        json_output[key] = None

# Print output
json_pretty = json.dumps(json_output, indent=4)
print(json_pretty)

{
    "summary": "The customer called to inquire about a specific type of printer and mentioned an order number. The customer was initially in the wrong department and expressed frustration about being on hold previously and getting disconnected. The agent, Valerie, offered to connect the customer to the correct department and also offered a $20 gift card for the trouble, which the customer declined. The call ended with the customer being placed on hold due to all agents being busy.",
    "orderNumber": "806185814326",
    "productSKU": null,
    "contactType": "Phone",
    "productSafetyFlag": false,
    "customerNeed": "Technical or Product support",
    "employeeResponse": "Referred to other team - unable to help",
    "customerSentimentGoingIn": {
        "sentiment": "Negative",
        "reasoning": "The customer expressed frustration about being in the wrong department and having been on hold before getting disconnected."
    },
    "customerSentimentGoingOut": {
        "sentime